In [2]:
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
import pandas as pd

geom_file="/Users/afox/Jedi/Jedi_experiments/input/wrf_hydro_nwm_files/conus/geometry_cfs_long_range_ens.nc"
ncfile = netCDF4.Dataset(geom_file, mode='r')

# Get metadata from the file
npx = ncfile.dimensions["west_east"].size
npy = ncfile.dimensions["south_north"].size
npz = 1

lons = ncfile.variables["XLONG"][:]
lats = ncfile.variables["XLAT"][:]

# Print field dimensions
print(" Grid dimensions", npx, 'x', npy, 'x', npz)

# Zero out array to fill with field
hgt = np.zeros((npy, npx))

hgt[:,:] = ncfile.variables["HGT"][:,:]

 Grid dimensions 4608 x 3840 x 1


In [8]:
#obs_file = '/Users/afox/Jedi/Jedi_experiments/OWP_obs/wdb0_obs_swe_snwd_target_2020030100_p24h_m24h_max_sep_6h_test02.csv'
obs_file = '/Users/afox/Jedi/Jedi_experiments/Single_obs/taylor_park_obs.csv'
df = pd.read_csv(obs_file)
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,150.0,5.00,0,3828,3828,39.0005,-106.6142,2019-03-01T01:00:00Z,snow_water_equivalent_mm
1,99999,XXXX1,0.6,0.04,0,3828,3828,39.0005,-106.6142,2019-03-01T01:00:00Z,snow_depth_m
2,99998,XXXX2,150.0,5.00,0,3828,3828,38.8965,-106.5320,2019-03-01T01:00:00Z,snow_water_equivalent_mm
3,99998,XXXX2,0.6,0.04,0,3828,3828,38.8965,-106.5320,2019-03-01T01:00:00Z,snow_depth_m
4,99997,XXXX3,150.0,5.00,0,3828,3828,38.9234,-106.7456,2019-03-01T01:00:00Z,snow_water_equivalent_mm
5,99997,XXXX3,0.6,0.04,0,3828,3828,38.9234,-106.7456,2019-03-01T01:00:00Z,snow_depth_m


In [9]:
latitude = df['latitude']
longitude = df['longitude']

nobs = np.shape(longitude)
nobs = nobs[0]

In [10]:
lon_req = longitude
lat_req = latitude

array1 = np.asarray(lons)
array2 = np.asarray(lats)

height_out = np.zeros(nobs)

for i in range(nobs):
    lons_diff = (array1 - (lon_req[i]))**2
    lats_diff = (array2 - (lat_req[i]))**2
    comb_diff = lons_diff+lats_diff
    idx = comb_diff.argmin()
    height_out[i] = hgt.flatten()[idx]

In [11]:
df['dem_elev_m']=height_out
df['rec_elev_m']=height_out
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,150.0,5.00,0,3828.214844,3828.214844,39.0005,-106.6142,2019-03-01T01:00:00Z,snow_water_equivalent_mm
1,99999,XXXX1,0.6,0.04,0,3828.214844,3828.214844,39.0005,-106.6142,2019-03-01T01:00:00Z,snow_depth_m
2,99998,XXXX2,150.0,5.00,0,3013.291992,3013.291992,38.8965,-106.5320,2019-03-01T01:00:00Z,snow_water_equivalent_mm
3,99998,XXXX2,0.6,0.04,0,3013.291992,3013.291992,38.8965,-106.5320,2019-03-01T01:00:00Z,snow_depth_m
4,99997,XXXX3,150.0,5.00,0,3569.580078,3569.580078,38.9234,-106.7456,2019-03-01T01:00:00Z,snow_water_equivalent_mm
5,99997,XXXX3,0.6,0.04,0,3569.580078,3569.580078,38.9234,-106.7456,2019-03-01T01:00:00Z,snow_depth_m


In [23]:
df.to_csv (obs_file, index = False, header=True)